# Die Webseite der TH Nürnberg- Intranet Scrapen

### Einleitung

Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

Conventions: 
- pandas Spaltennamen im Sigular
- Die meißten Links sind keine URLs, da sie lokal sind

In [ ]:
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
from db_init import db_get_df, db_save_df
import glob
import json
import requests
import sys
from tqdm import tqdm

### Scrapen der home Seite

Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite der TH-Nürnberg. (https://www.th-nuernberg.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite der TH verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.

Eine Funktion, die eine URL als parameter nimmt und das HTML file zurückgibt, falls die Seite existiert.

Dafür nutzen wir die requests Bibliothek.

In [ ]:
def download_html_from_url(url):
    res = requests.get(url)
    html = ""
    if res.ok:
        html =  res.text
    return html

Eine Funktion, die ein HTML file nach Links durchsucht und alle gefundenen externen und internen Links zurückgibt.
Dafür nutzen wir die Bibliothek Beautifulsoup, mit dem lxml parser.

In [ ]:
def get_links_from_one_html(html):
    soup = BeautifulSoup(html,"lxml")
    links = [a["href"] for a in soup.find_all('a', href=True)]
    return links

Jetzt geben wir unsere initiale URL an und extrahieren alle Links aus dieser

In [ ]:
BASE_URL = "https://intern.ohmportal.de/"
html = download_html_from_url(BASE_URL)
links = get_links_from_one_html(html)
df = pd.DataFrame({"link": links})
print(*df["link"])
print(len(df["link"]))

### Links filtern

Zunächst können wir alle Links überprüfen, ob sie Parameter oder sections mit 
- *?param1=hallo*
- *#section* 

enthalten. Beide Attribute sind für den Download der Webseiten nicht notwendig und werden ausgefiltert. Dies spart uns HTML Duplikate.

In [ ]:
def clean_links(links):
    cleaned_links = [link.split('#')[0].split('?')[0] for link in links]
    return cleaned_links

Nun speichern wir das zwischen Ergebnis der gefilterten Links.

In [ ]:
df["link"]=clean_links(df["link"])
print(*df["link"])


Nun löschen wir alle Links, die kein Inhalt haben (leere Links). 

In [ ]:
def remove_empty_links(links):
    cleaned_links = [link for link in links if link.strip()]
    return cleaned_links

In [ ]:
df = pd.DataFrame({"link": remove_empty_links(df["link"])})
print(*df["link"])
print(len(df["link"]))

Man sieht, dass wir 7 Links ausgefiltert haben, wenn man die Anzahl der Links vor und nach der Filterung vergleicht.

### Externe Links finden

Jetzt können wir Mal nachschauen, auf welche externen Seiten die Startseite der THN verweist.

In [ ]:
def find_extern_urls(urls):
    external_links = []
    for link in urls:
        if link.startswith("http"):
            external_links.append(link)

    return external_links

In [ ]:
external_links = find_extern_urls(df["link"])
print("Anzahl externer Links: ", len(external_links))
print(external_links)

Die Seite verweist also auf alle gängigen Sozial Media Seiten, wie Twitter, youtube, tiktok (der nicht existiert), instagram (der nicht existiert), xing, oft auf die jobbörse mit mehreren Links ins Intranet und auf die Efi fakultät,

### Interne Links filtern

Wir filtern nun noch alle Links heraus, die keine HTML-inhalte besitzen, wie z.B. pdf oder xml Dateien, die mail Links enthalten oder die nicht auf die THN Webseite referieren.

In [ ]:
def filter_intern_links(urls):
    filtered_links = []
    for url in urls:
        if url.startswith("http"):
            continue
        elif url.startswith("mailto:"):
            continue
        elif url.startswith("javascript:"):
            continue
        elif url.startswith("&#"): # is encoded mailto
            continue
        elif ".xml" in url:
            continue
        elif ".pdf" in url:
            continue
        elif url == "/":
            continue
        elif url == "&":
            continue
        else:
            filtered_links.append(url)
    return filtered_links

In [ ]:
intern_links = filter_intern_links(df["link"])
print("Anzahl interner Links: ", len(intern_links))
print(intern_links)

Nun haben wir weitere 55 Links entfernt.

### Dublikate entfernen und sortieren

Jetzt können wir die duplikate entfernen und anschließend alphabetisch sortieren.

In [ ]:
def sort_and_remove_dublicates(df):
    df = df.sort_values("link")
    df = df.drop_duplicates(subset="link")
    df = df.reset_index(drop=True)
    return df

In [ ]:
intern_links = filter_intern_links(df["link"])
df = pd.DataFrame({"link": intern_links})
df = sort_and_remove_dublicates(df)
print("Anzahl interner Links (ohne Dublikate): ", len(intern_links))
print(*df["link"])

Anhand der Anzahl der Links vor und nach der Duplikaten Entfernung, sieht man das die THN Startseite, keine doppelt vorkommende Links enthält.

### Links abspeichern

Für unsere weiteren Schritte werden wir immer nur interne Links verwenden, deshalb speichern wir an dieser Stelle mal die internen Links ab.

In [ ]:
db_save_df(df, "only_links")

### Downloaden der files

Jetzt können wir mit dem downloaden anfangen.

Diese Funktion lädt nun alle Html files zu den Links herunter und speichert sie im Dataframe neben den "link" in einer Spalte "html".

In [ ]:
def download_all_urls(links):
    htmls = []
    for link in tqdm(links):
        url = "https://www.th-nuernberg.de" + link
        html = download_html_from_url(url)
        htmls.append(html)
    return htmls

In [ ]:
df["html"]=download_all_urls(df["link"])

Wir können die Daten an dieser Stelle abspeichern.

In [ ]:
db_save_df(df, "intranet_html_iter_01")

### Weitere Iterationsstufen

Wenn wir ab diesem Abschnitt starten können wir die vorher gesammelten Daten neu laden.

In [ ]:
df = db_get_df("intranet_html_iter_01")

Jetzt können wir die heruntergeladenen HTML files nach weiteren Links durchsuchen und Sie dem Dataframe hinzufügen

In [ ]:
def find_all_links_in_html(htmls):
    all_links = []
    for html in tqdm(htmls):
        links = get_links_from_one_html(html)
        links= clean_links(links)
        links= remove_empty_links(links)
        links= filter_intern_links(links)
        [all_links.append(link) for link in links]
    return all_links

In [ ]:
all_links = find_all_links_in_html(df["html"])
len(all_links)

Wir haben jetzt also 296 Links gesammelt, davon sind aber viele Dublikate.

In [ ]:
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])

In [ ]:
df = sort_and_remove_dublicates(df)
len(df["link"])

Gefiltert nach dublikaten haben wir nun also noch 277 Links

In [ ]:
print(*df["link"])

### Iteratives Downloaden

Um den Daten nun weitere Webseiten hinzuzufügen, können wir für jede weitere URL schauen, ob sie schon heruntergeladen wurde. Wenn nicht, dann laden wir sie jetzt herunter.

In [ ]:
def update_df_with_html(df):
    for index, row in tqdm(df.iterrows()):
        if pd.isna(row['html']) or row['html'] == '':
            url = "https://intern.ohmportal.de/" + row["link"]
            html = download_html_from_url(url)
            df.at[index, 'html'] = html
    
    return df

In [ ]:
print(df.loc[2])


In [ ]:
df = update_df_with_html(df)

### Iterationen


1. Iteration: Startseite THN (~300 Links)
2. Iteration: Links von startseite aufrufen und HTML scrapen (~2000 Links)
3. Iteration: Links von diesen Seiten aufrufen und HTML scrapen (~ Links)

In [ ]:
def iteration(df):
    all_links = find_all_links_in_html(df["html"])
    df_new = pd.DataFrame({"link": all_links, "html": None})
    df = pd.concat([df, df_new])
    df = sort_and_remove_dublicates(df)
    df=update_df_with_html(df)
    return df

In [ ]:
df=iteration(df)

### Texte extrahieren

Als nächstes müssen wir aus den rohen HTML Dokumenten die unrelevanten Daten aussortieren

In [ ]:
df = db_get_df("html_iter_01")

Die nachfolgende Funktion bestimmt, ob ein Beautifulsoup geparstes HTML Element sichtbar ist oder nicht.

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

Jetzt bestimmen wir eine Funktion, die ein HTML als Input bekommt und daraus die Texte und Titeln bestimmt.

Die HTML Seiten haben 3 verschiedene Strukturen:

- "main" (https://www.th-nuernberg.de/hochschule-region/organisation-und-struktur/hochschulleitung-und-gremien/)
- "div", {'class': 'portal'} (https://www.th-nuernberg.de/fakultaeten/bi/)
- "div", {'class': 'page-wrap'} (https://www.th-nuernberg.de/studium-karriere/studien-und-bildungsangebot/duale-studienmodelle/studium-mit-vertiefter-praxis/)

In [ ]:
def get_content(file):
    soup = BeautifulSoup(file,"lxml")

    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""

    main = soup.find("main")
    portal = soup.find("div", {'class': 'portal'})
    page_container = soup.find("div", {'class': 'page-wrap'})
    
    visible_texts = ""
    if main:
        container = main.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)

    elif portal:
        texts = portal.find_all(text=True)
        visible_texts = filter(tag_visible, texts)

    elif page_container:
        container = page_container.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)

    return {
        "title":    title,
        "text":     u" ".join(t.strip() for t in visible_texts)
    }

Nun speichern wir die Texte sowie die dazugehörigen Titeln in den Dataframe.

In [ ]:
parsed_texts = []
titles = []
for html in tqdm(df["html"]):
    content = get_content(html)
    parsed_texts.append(content["text"])
    titles.append(content["title"])

df["text"] = parsed_texts
df["title"] = titles

In [ ]:

print(df[df["text"] != ""]["url"])

In [ ]:
db_save_df(df, "html_attribute")